In [3]:
import pandas as pd
import numpy as np
import uuid
import os 
from collections import defaultdict
from datetime import date
import datetime
from pandasql import sqldf
import matplotlib.pyplot as plt

# File Import and DF Creation

In [4]:
# Function imports file and creates df with additional column needed for analysis
def import_file(file_path):
    
    #Move directories to find file
    os.getcwd()
    os.chdir('../data')

    
    #Read excel file and drop unnecessary column
    data = pd.read_excel(file_path)
    data.drop('Unnamed: 0', axis=1, inplace = True)
    #Add the 'course_term' column to df
    data['course_term'] = data['unique_course'] + data['enrl_term_id'].astype(str)
    
    # Remove any Spring 2020 P grades as these are not reliable
    data['remove'] = data[['enrl_term_id', 'CRS_GRADE']].apply(lambda x: True if (x['enrl_term_id'] == 202001 and x['CRS_GRADE'] == 'P') else False, axis = 1)
    data_filtered = data.loc[data["remove"] != True].copy()
    data_filtered.drop('remove', axis=1, inplace=True)
    
    return data_filtered


#Function creates two dfs that will be used for course identification and student historical file creation
def create_dfs(df):
    
    #Exclude 202108 because at the time file was received, grades for Fall 2021 had not been released
    semester_list = [201901, 201905, 201908, 202001, 202005, 202008, 202101, 202105]
    filtered_df = df.loc[df['enrl_term_id'].isin(semester_list)]
    
    #Create a grouping by course to identify high enrollment courses
    crs_groups = filtered_df.groupby(['unique_course']).agg({'uuid': ['nunique']})
    crs_groups.columns = crs_groups.columns.droplevel(0)
    crs_groups = crs_groups.reset_index(inplace=False)
    
    #Sort the above df and keep only the first 25 courses (high enrollment) 
    crs_groups.sort_values(by=['nunique'], inplace=True, ascending=False)
    high_enrollment = crs_groups.head(25)
    
    #Create a simplified df with only the columns that strictly necessary to identify course-term-student 
    #combinations
    data_simplified = df[['uuid', 'unique_course', 'enrl_term_id', 'course_term', 'crs_avg_grd_term', 
                          'CRS_GRADE', 'grade_point_value']]
    
    return filtered_df, high_enrollment, data_simplified 


# Helper function to help calculate the historical grade avg per course
def course_avg(avg_grade, terms):
    
    return avg_grade / terms


# Helper function to help calculate the pass/fail status of a student/course pairing
def pf_flag(grade): 
    if grade in ['A', 'A-', 'AU', 'B', 'B+', 'B-', 'C', 'C+', 'C-', 'P']: 
        return 'pass'
    return 'fail'

# Helper function to get year from term
# This works because python uses floor division (will always round down to nearest int, giving me right year)
def remove_last_two(num): 
    return (num // 100)


# Helper function to calculate the difference between two columns
# Used to calculate the years elapsed
def difference(a,b):
    return a - b

# Helper function to flag the row containing the latest term in which a course is taken    
def keep_latest(enrl_term, latest_term):
    is_latest = False
    if enrl_term == latest_term:
        is_latest = True
    return is_latest


# Function to calculate UCC data
def calc_ucc(df):
    
    # Create a df to hold only those rows with UCC courses
    filtered = df.loc[df['UCC_CRSE_FLG'] == 'Y']
    
    # Find the latest term the UCC course was taken in (use this to determine pass/fail)
    latest_term_UCC = filtered.groupby(['uuid', 'unique_course']).agg({'enrl_term_id': ['max']}).rename(columns={'max':'latest_term'})
    latest_term_UCC.columns = latest_term_UCC.columns.droplevel(0)
    latest_term_UCC = latest_term_UCC.reset_index(inplace=False)

    # Merge into filtered file with all data (pertaining to only UCC courses)
    UCC_latest = pd.merge(filtered, latest_term_UCC, how='left', on=['uuid', 'unique_course'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    # Create attribute to identify the row with the latest grade for UCC course (based on latest term)
    UCC_latest['is_latest_grade']  = UCC_latest.apply(lambda x: keep_latest(x['enrl_term_id'],x['latest_term']), axis=1)

    # Create a df with UCC Courses and only the latest grades
    UCC_ = UCC_latest.loc[UCC_latest['is_latest_grade'] == True]
    
    # Calculate the # of UCCs taken
    taken_UCC = UCC_.groupby(['uuid']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'UCCs_taken'})
    taken_UCC.columns = taken_UCC.columns.droplevel(0)
    taken_UCC = taken_UCC.reset_index(inplace=False)
    
    # Calculate pass/fail for UCCs
    Pass_Fail_UCC = UCC_.groupby(['uuid', 'Pass_Fail']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'unique_courses'})
    Pass_Fail_UCC.columns = Pass_Fail_UCC.columns.droplevel(0)
    Pass_Fail_UCC = Pass_Fail_UCC.reset_index(inplace=False)
    
    # Pivot rows to columns to prepare for merging to main file
    pivoted_UCC_PF = Pass_Fail_UCC.pivot(index='uuid', columns='Pass_Fail', values='unique_courses').reset_index()
    pivoted_UCC_PF.columns.name=None
    pivoted_UCC_PF.rename(columns = {'fail': 'UCCs_failed', 'pass':'UCCs_passed'}, inplace = True)
    
    
    # Merge data back into main dataframe
    merge_1 = pd.merge(df, taken_UCC, how='left', on=['uuid'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)

    main_UCC = pd.merge(merge_1, pivoted_UCC_PF, how='left', on=['uuid'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    return main_UCC
    
    
    
# Function to calculate gateway data
def calc_gateway(df):
    
    # Create a df to hold only those rows with gateway courses
    filtered = df.loc[df['GATEWAYCRSE_FLG'] == 'Y']
    
    # Find the latest term the gateway course was taken in (use this to determine pass/fail)
    latest_term_gate = filtered.groupby(['uuid', 'unique_course']).agg({'enrl_term_id': ['max']}).rename(columns={'max':'latest_term'})
    latest_term_gate.columns = latest_term_gate.columns.droplevel(0)
    latest_term_gate = latest_term_gate.reset_index(inplace=False)

    # Merge into filtered file with all data (pertaining to only gateway courses)
    gateway_latest = pd.merge(filtered, latest_term_gate, how='left', on=['uuid', 'unique_course'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    # Create attribute to identify the row with the latest grade for gateway course (based on latest term)
    gateway_latest['is_latest_grade']  = gateway_latest.apply(lambda x: keep_latest(x['enrl_term_id'],x['latest_term']), axis=1)

    # Create a df with gateway Courses and only the latest grades
    gateway_ = gateway_latest.loc[gateway_latest['is_latest_grade'] == True]
    
    # Calculate the # of gateways taken
    taken_gate = gateway_.groupby(['uuid']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'gateways_taken'})
    taken_gate.columns = taken_gate.columns.droplevel(0)
    taken_gate = taken_gate.reset_index(inplace=False)
    
    # Calculate pass/fail for gateways
    Pass_Fail_gate = gateway_.groupby(['uuid', 'Pass_Fail']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'unique_courses'})
    Pass_Fail_gate.columns = Pass_Fail_gate.columns.droplevel(0)
    Pass_Fail_gate = Pass_Fail_gate.reset_index(inplace=False)
    
    # Pivot rows to columns to prepare for merging to main file
    pivoted_gate_PF = Pass_Fail_gate.pivot(index='uuid', columns='Pass_Fail', values='unique_courses').reset_index()
    pivoted_gate_PF.columns.name=None
    pivoted_gate_PF.rename(columns = {'fail': 'gateways_failed', 'pass':'gateways_passed'}, inplace = True)
    
    
    # Merge data back into main dataframe
    merge_1 = pd.merge(df, taken_gate, how='left', on=['uuid'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)

    main_gateway = pd.merge(merge_1, pivoted_gate_PF, how='left', on=['uuid'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    return main_gateway 


# Function to calculate general pass/fail counts per term
def calc_pf_term(df):
    
    #Create a temporary dataframe
    temp2 = df.copy()
    
    # Calculate the # of courses taken per term
    taken_by_term = temp2.groupby(['uuid', 'enrl_term_id']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'term_courses_taken'})
    taken_by_term.columns = taken_by_term.columns.droplevel(0)
    taken_by_term = taken_by_term.reset_index(inplace=False)

    # Calculate the number of courses passed vs failed in term
    Pass_Fail_term = temp2.groupby(['uuid','enrl_term_id' ,'Pass_Fail']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'unique_courses'})
    Pass_Fail_term.columns = Pass_Fail_term.columns.droplevel(0)
    Pass_Fail_term = Pass_Fail_term.reset_index(inplace=False)
    
    # Pivot rows to columns to prepare for merging to main file
    pivoted_term = Pass_Fail_term.pivot(index=['uuid', 'enrl_term_id'], columns='Pass_Fail', values='unique_courses').reset_index()
    pivoted_term.columns.name=None
    pivoted_term.rename(columns = {'fail': 'term_courses_failed', 'pass':'term_courses_passed'}, inplace = True)

    # Merge attributes back into main dataframe
    merge_term_1 = pd.merge(df, taken_by_term, how='left', on=['uuid', 'enrl_term_id'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)

    final = pd.merge(merge_term_1, pivoted_term, how='left', on=['uuid', 'enrl_term_id'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    return final


# Function to calculate general pass/fail counts overall
def calc_pf(df):
    
    #Create a temporary dataframe
    temp = df.copy()
    
    # Find the latest term the course was taken in (use this to determine pass/fail)
    latest_term_all = temp.groupby(['uuid', 'unique_course']).agg({'enrl_term_id': ['max']}).rename(columns={'max':'latest_term'})
    latest_term_all.columns = latest_term_all.columns.droplevel(0)
    latest_term_all = latest_term_all.reset_index(inplace=False)
    
    # Merge into filtered file with all data (pertaining to all courses)
    all_latest = pd.merge(temp, latest_term_all, how='left', on=['uuid', 'unique_course'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    # Create attribute to identify the row with the latest grade for course (based on latest term)
    all_latest['is_latest_grade']  = all_latest.apply(lambda x: keep_latest(x['enrl_term_id'],x['latest_term']), axis=1)
    
    # Create a df with all Courses and only the latest grades
    all_ = all_latest.loc[all_latest['is_latest_grade'] == True]
    
    # Calculate the # of courses taken
    taken_ = all_.groupby(['uuid']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'total_courses_taken'})
    taken_.columns = taken_.columns.droplevel(0)
    taken_ = taken_.reset_index(inplace=False)
    
    # Calculate the number of courses passed vs failed
    Pass_Fail_all = all_.groupby(['uuid', 'Pass_Fail']).agg({'unique_course': ['nunique']}).rename(columns={'nunique':'unique_courses'})
    Pass_Fail_all.columns = Pass_Fail_all.columns.droplevel(0)
    Pass_Fail_all = Pass_Fail_all.reset_index(inplace=False)

    # Pivot rows to columns to prepare for merging to main file
    pivoted_all = Pass_Fail_all.pivot(index='uuid', columns='Pass_Fail', values='unique_courses').reset_index()
    pivoted_all.columns.name=None
    pivoted_all.rename(columns = {'fail': 'total_courses_failed', 'pass':'total_courses_passed'}, inplace = True)
    
    # Merge data back into main dataframe
    main_ = pd.merge(df, pivoted_all, how='left', on=['uuid'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    return main_


# Removes duplicate course/grade instances per student and keeps only latest grade per course
def remove_retakes (df_course):
    
    #Make copy of df_course
    course_data = df_course.copy()
    
    # Find the latest term the course was taken in (use this to remove past retakes)
    latest_term = df_course.groupby(['uuid', 'unique_course']).agg({'enrl_term_id': ['max']}).rename(columns={'max':'latest_term'})
    latest_term.columns = latest_term.columns.droplevel(0)
    latest_term = latest_term.reset_index(inplace=False)

    # Merge into file with all data
    all_ = pd.merge(course_data, latest_term, how='left', on=['uuid', 'unique_course'],
         suffixes=('_og', '_stu'), copy=True, indicator=False,
         validate=None)
    
    # Create attribute to identify the row with the latest grade for course (based on latest term)
    all_['is_latest_grade']  = all_.apply(lambda x: keep_latest(x['enrl_term_id'],x['latest_term']), axis=1)

    # Create a df with Courses and only the latest grades
    latest_only = all_.loc[all_['is_latest_grade'] == True]
    
    return latest_only
        
    
# Creates the course attributes for analysis for train data
def calc_crs_vars_train(df_data, df_course):    
    
    # Identify Course/term pairings found in train data
    crs_term = df_course.groupby(['unique_course', 'enrl_term_id'], as_index=False).agg({'uuid':['nunique']})
    crs_term.columns = crs_term.columns.get_level_values(0)
    
    # Create list of terms
    term_list = crs_term['enrl_term_id'].tolist()
    
    # Create list to house course vars "arrays"
    crs_vars_train = []
    
    for term in term_list:
    
        # Filter df to include only info prior to specified term
        filtered = df_data.loc[((df_data['enrl_term_id'] <= term) & (df_data['unique_course'] == df_course['unique_course'].values[0]))] 

        # Group df by course, term and crs_avg_grade
        df_1 = filtered.groupby(['unique_course', 'enrl_term_id','crs_avg_grd_term'], as_index=False).agg({'uuid':['nunique']})
        df_1.columns = df_1.columns.get_level_values(0)

        # Sum total course avg grades and count number of terms course has been taught
        df_2 = df_1.groupby(['unique_course'], as_index = False).agg({'enrl_term_id':['nunique'], 'crs_avg_grd_term':['sum']})
        df_2.columns = df_2.columns.get_level_values(0)

        # Calculate avg course grade historically (only accounting for data included in filtered df)
        df_2['crs_avg_grade_all'] = df_2[['enrl_term_id', 'crs_avg_grd_term']].apply(lambda x: course_avg(x['crs_avg_grd_term'], x['enrl_term_id']), axis = 1)
        df_2 = df_2.rename(columns={'enrl_term_id': 'tot_terms_tgt'})
        df_2.drop(['crs_avg_grd_term'], axis = 1, inplace = True)
        
        # Create 'array' and append to list
        row = [df_2['unique_course'].values[0], term, df_2['tot_terms_tgt'].values[0], df_2['crs_avg_grade_all'].values[0]]
        #Append to main list_course_dets
        crs_vars_train.append(row)
        
    
    #return df with crs_avg_grade_all as well as tot_terms_taught
    final_vars = pd.DataFrame(crs_vars_train, columns = ['unique_course', 'enrl_term_id', 'tot_terms_tgt',
                                                    'crs_avg_grade_all'])
    
    
    return final_vars


# Creates the course attributes for analysis
def calc_crs_vars(df, term, course):
    
    #Filter df to include only info prior to specified term
    filtered = df.loc[((df['enrl_term_id'] <= term)) & (df['unique_course'] == course)] 
    
    # Group df by course, term and crs_avg_grade
    df_1 = filtered.groupby(['unique_course', 'enrl_term_id','crs_avg_grd_term'], as_index=False).agg({'uuid':['nunique']})
    df_1.columns = df_1.columns.get_level_values(0)
    
    # Sum total course avg grades and count number of terms course has been taught
    df_2 = df_1.groupby(['unique_course'], as_index = False).agg({'enrl_term_id':['nunique'], 'crs_avg_grd_term':['sum']})
    df_2.columns = df_2.columns.get_level_values(0)
    
    # Calculate avg course grade historically (only accounting for data included in filtered df)
    df_2['crs_avg_grade_all'] = df_2[['enrl_term_id', 'crs_avg_grd_term']].apply(lambda x: course_avg(x['crs_avg_grd_term'], x['enrl_term_id']), axis = 1)
    df_2 = df_2.rename(columns={'enrl_term_id': 'tot_terms_tgt'})
    df_2.drop(['crs_avg_grd_term'], axis = 1, inplace = True)
    
    #return df with crs_avg_grade_all as well as tot_terms_taught
    return df_2
    
    
# Creates the student attributes for analysis    
def calc_stu_vars(df):
    
    # Group df by student, term and creds attempted
    df_1 = df.groupby(['uuid', 'enrl_term_id','creds_attp_term'], as_index=False).agg({'unique_course':['nunique']})
    df_1.columns = df_1.columns.get_level_values(0)
    
    # Sum total creds earned and count number of terms student has been enrolled
    df_2 = df_1.groupby(['uuid'], as_index=False).agg({'enrl_term_id':['nunique'], 'creds_attp_term':['sum']})
    df_2.columns = df_2.columns.get_level_values(0)
    df_2 = df_2.rename(columns={'enrl_term_id': 'tot_terms_enrled', 'creds_attp_term':'creds_attp_all'})
    
    #return df with total creds earned and total terms enrolled per student
    return df_2

# Creates enrollment-based attributes for analysis
def calc_enrl_vars(df):
  
    # Create new attribute based on pass/fail function that uses course grade
    df['Pass_Fail'] = df['CRS_GRADE'].apply(pf_flag)
    
    #Find the minimun enrollment term per uuid, keep only relevant columns to then merge back to main df
    #Create temp df to save results
    min_enrl_term = df.loc[df.groupby('uuid')['enrl_term_id'].idxmin()][['uuid', 'enrl_term_id']]
    
    #Rename columns
    min_enrl_term = min_enrl_term.rename(columns={'enrl_term_id': 'min_enrl_term'})
    
    # Add minimum enrollment term to main file
    df_min = pd.merge(df, min_enrl_term, how='left', on=['uuid'],
         suffixes=('_og', '_min'), copy=True, indicator=False,
         validate=None)
    
    #Find the maximum enrollment term per EMPLID, keep only relevant columns to then merge back to main df
    #Create temp df to save results
    max_enrl_term = df_min.loc[df_min.groupby('uuid')['enrl_term_id'].idxmax()][['uuid', 'enrl_term_id']]
    
    #Rename columns
    max_enrl_term = max_enrl_term.rename(columns={'enrl_term_id': 'max_enrl_term'})
    
    # Add maximum enrollment term to main file
    df_max = pd.merge(df_min, max_enrl_term, how='left', on=['uuid'],
         suffixes=('_og', '_min'), copy=True, indicator=False,
         validate=None)
    
    #Create an minimum enrollment year column to calculate elapsed time
    df_max['min_enrl_year'] = df_max['min_enrl_term'].apply(remove_last_two)
    
    # Create a maximum enrollment year column to calculate elapsed time
    df_max['max_enrl_year'] = df_max['max_enrl_term'].apply(remove_last_two)
    
    # Create time_elapsed attribute by finding difference between min and max enrollment years
    df_max['time_elapsed']  = df_max.apply(lambda x: difference(x['max_enrl_year'],x['min_enrl_year']), axis=1)
    
    #Calculate UCC-related vars
    df_ucc = calc_ucc(df_max)
    
    #Calculate gateway-related vars
    df_gateway = calc_gateway(df_ucc)
    
    #Calculate general pass/fail
    df_pf = calc_pf(df_gateway)
    
    return df_pf

# Identify terms available for Course & Create Student Historical Files 

In [ ]:
# Create dataframe with chosen courses, latest available term, second to last available term
# and previous term (in relation to second to last term)
# Returns dictionaries of dataframes with student uuids for each course/term
def create_course_details(course_summ_df, years_df, data_file):
    
    course_list = course_summ_df['unique_course'].tolist()
    
    list_course_dets = []
    latest = {}
    sectl = {} #second to last
    thirdtl = {} #third to last
    
    for course in course_list:
        
        # Create a filtered df per course
        course_df = years_df.loc[years_df['unique_course'] == course]
        
        # Create a summary df of the terms this course was taught (based on the enrollments)
        terms_summ_df = course_df.groupby(course_df['enrl_term_id']).agg({'uuid': ['nunique']})
        terms_summ_df.columns = terms_summ_df.columns.droplevel(0)
        terms_summ_df = terms_summ_df.reset_index(inplace=False)
        
        # Calculate the "rank" of each enrl_term_id to determine the last three terms in data per course
        terms_summ_df['rank'] = terms_summ_df['enrl_term_id'].rank()
        
        #Check that max_rank is >= 3
        max_rank = terms_summ_df['rank'].max()
        
        if max_rank < 3:
            continue
        else:
            #Find latest term
            latest_term = terms_summ_df.loc[terms_summ_df['rank'] == max_rank]['enrl_term_id'].values[0]
            
            #Find second to last term (in relation to latest term)
            sec_to_last_term = terms_summ_df.loc[terms_summ_df['rank'] == max_rank - 1]['enrl_term_id'].values[0]
            #Find previous term (prev in relation to second to last term)
            prev_term = terms_summ_df.loc[terms_summ_df['rank'] == max_rank - 2]['enrl_term_id'].values[0]
            
            #Creating df per course for each term (latest, second to latest, prev term)
            latest_df = data_file.loc[(data_file['unique_course'] == course) & 
                                (data_file['enrl_term_id'] == latest_term)]
            stl_df = data_file.loc[(data_file['unique_course'] == course) & 
                                (data_file['enrl_term_id'] == sec_to_last_term)]
            prev_df = data_file.loc[(data_file['unique_course'] == course) & 
                                (data_file['enrl_term_id'] <= prev_term)] 
            
            
            #Remove retakes from train data
            prev_df_nr = remove_retakes(prev_df)
            
            #Calculate # of times course taught & course avg grade historical
            late = calc_crs_vars(data_file, latest_term, course)
            slate = calc_crs_vars(data_file, sec_to_last_term, course)
            tlate = calc_crs_vars_train(data_file, prev_df_nr) 
                
            #Merge into df created per course above
            latest_final = pd.merge(latest_df, late[['unique_course','tot_terms_tgt', 'crs_avg_grade_all']], 
                                    how='left', on=['unique_course'], suffixes=('_og', '_stu')
                                    ,copy=True, indicator=False,validate=None)

            stl_final = pd.merge(stl_df, slate[['unique_course','tot_terms_tgt', 'crs_avg_grade_all']], 
                                    how='left', on=['unique_course'], suffixes=('_og', '_stu')
                                    ,copy=True, indicator=False,validate=None)
            
            prev_final = pd.merge(prev_df_nr, tlate[['unique_course','enrl_term_id','tot_terms_tgt', 'crs_avg_grade_all']], 
                                    how='left', on=['unique_course', 'enrl_term_id'], suffixes=('_og', '_stu')
                                    ,copy=True, indicator=False,validate=None)
            
            #Remove unwanted columns from train data df
            prev_final.drop(['is_latest_grade','latest_term'], axis = 1, inplace = True)
            
            #Appending the dfs to a dictionary (according to term snapshot)
            latest[course] = latest_final
            sectl[course] = stl_final
            thirdtl[course] = prev_final
            
            #Create temp list of course details  
            course_dets = [course, latest_term, sec_to_last_term, prev_term]
            #Append to main list_course_dets
            list_course_dets.append(course_dets)
        
    
    data = pd.DataFrame(list_course_dets, columns = ['Unique Course', 'Latest Term', 'Second to Last Term',
                                                    'Third from Last Term'])
    
    return data, latest, sectl, thirdtl
     
    

# This function will take as input four params and create a dictionary 
# to house dfs with student historical data
'''
param1: This is the list of courses that were identified as having high enrollment
param2: This is the dictionary that corresponds to the set of data that will be examined
param3: This is the df housing the full data set from which historical data will be extracted
'''
def collect_student_hist(high_enrl_crs, crs_dict, full_data_file):
    
    course_list = high_enrl_crs['Unique Course'].tolist()
    
    student_hist_data = {}
    
    for course in course_list:
        
        #Get term that will serve as limit for obtaining student historical data
        #term = crs_dict[course]['enrl_term_id'].values[0]
        
        #Create empty df to save student files per term
        #students = pd.DataFrame(columns = list(full_data_file.columns.values))
        students = pd.DataFrame()
        
        #Get list of terms that course was taught in
        term_list = crs_dict[course]['enrl_term_id'].unique().tolist()
        
        for term in term_list:
            #print(course, " ", term)
            #Create copy of df that holds the information for this specific course/term pairing
            course_df = crs_dict[course].copy()
            #print(course_df)
            course_filtered = course_df.loc[course_df['enrl_term_id'] == term]
            
            course_final = course_filtered[['uuid','unique_course','tot_terms_tgt','crs_avg_grd_term', 'crs_avg_grade_all', 
                                   'CRS_GRADE', 'grade_point_value']]
            course_final = course_final.rename(columns={'unique_course': 'target_crs', 'tot_terms_tgt':'target_terms_tgt',
                                     'crs_avg_grd_term':'target_avg_grd_term', 'crs_avg_grade_all':'target_avg_grade_all',
                                     'CRS_GRADE':'target_crs_grade', 'grade_point_value':'target_gpv'})

            #Get the list of students enrolled in the course (for specified term)
            students_enrl = course_final['uuid'].tolist()
            
                

            #Get the historical data for the students enrolled in term
            student_df = full_data_file.loc[(full_data_file['uuid'].isin(students_enrl)) & 
                                    (full_data_file['enrl_term_id'] < term) & 
                                    (full_data_file['unique_course'] != course)]
            
            if student_df.empty == False:
                
                #Calculate terms taken and total credits attempted and merge back to main file
                to_merge = calc_stu_vars(student_df)
                student_mid = pd.merge(student_df, to_merge[['uuid','tot_terms_enrled', 'creds_attp_all']], 
                                            how='left', on=['uuid'], suffixes=('_og', '_stu')
                                            ,copy=True, indicator=False,validate=None)
                
                #Keep only those students who have been enrolled for two terms or more
                student_mid_2 = student_mid.loc[student_mid['tot_terms_enrled'] >= 2].copy()
                
                if student_mid_2.empty == False:
                
                    #Calculate additional student course enrollment attributes

                    student_final = calc_enrl_vars(student_mid_2)

                    stu_crs = pd.merge(student_final, course_final, 
                                                how='left', on=['uuid'], suffixes=('_og', '_stu')
                                                ,copy=True, indicator=False,validate=None)

                    #Union dataframe with other terms' student lists
                    students = students.append(stu_crs)
                    students_noretakes = remove_retakes(students) 
                else: 
                    continue
            else:
                continue
            
        #Append df to dictionary
        student_hist_data[course] = students_noretakes
        
        
    return student_hist_data


# Function to create the historical enrollment files for students enrolled in course of interest
def create_student_files(courses_, tr, va, te):
    
    course_list = courses_['Unique Course'].tolist()
    
    for course in course_list:
        
        course_name = course.replace("-", "")

        with pd.ExcelWriter('files/{}.xlsx'.format(course_name)) as writer:
            train[course].to_excel(writer, sheet_name='train')
            validate[course].to_excel(writer, sheet_name='validate')
            test[course].to_excel(writer, sheet_name='test')

    return


# Execute code to identify high enrollment courses, find their latest, second to latest and third to latest terms
# Find the students enrolled in each of those terms
# Find each student's historical enrollment data and export to excel to use for model building
def run_script():
    # Import file
    full = import_file("anon_engvars_01232022.xlsx")
    print("Data imported")
    
    # Create the necessary dfs for analysis
    df_19_21, high_enrl_crs, simple_data = create_dfs(full)
    print("Dataframes created")
    
    # Create dictionaries of dataframes with student uuids for each course/term
    courses_dets, latest, stl, ttl = create_course_details(high_enrl_crs, df_19_21, simple_data)
    print("Dictionaries created")
        
    # Create subsets of data containing students previous terms enrollment
    train = collect_student_hist(courses_dets, ttl, full)
    validate = collect_student_hist(courses_dets, stl, full)
    test = collect_student_hist(courses_dets, latest, full)
    print("Train, Validate and Test subsets created")
    
    # Export historical enrollment files per course to excel
    create_student_files(courses_dets, train, validate, test)
    print("Historical enrollment files exported to excel!")
    
    print("Successful!")
    
    return full, df_19_21, high_enrl_crs

# Run Script and Create Files

In [19]:
 full, df_19_21, high_enrl_crs = run_script()

Data imported
Dataframes created
Dictionaries created
Train, Validate and Test subsets created
Historical enrollment files exported to excel!
Successful!


In [ ]:
full.info()

In [8]:
high_enrl_crs

,unique_course,nunique
1835,PSY-3211-,3176
1834,PSY-3024-,2963
1836,PSY-3215-,2413
1842,PSY-4931-,1900
1413,MAR-3023-,1830
748,ENC-1102-,1754
585,EAB-3002-,1723
1853,QMB-3200-,1692
468,COM-3112-,1685
1368,MAN-3025-,1656
